# Actividad (Parte 5) Explicabilidad usando contraejemplos
## Materia: Inteligencia Artificial Explicable - MSc. Inteligencia Artificial
Author: Esteban García-Cuesta, Departamento de Inteligencia Artificial, UPM (License CC-BY-NC)

Modificaciones hechas por Grupo 1:
<ul>
<li>Lidia Abad Azcutia</li>
<li>Sergio Arroni del Riego</li>
<li>David González Fernández</li>
</ul>

This code has been developed to be used exclusively for educational purposes.

## Introducción
La explicabilidad con contraejemplos permite conocer el funcionamiento del modelo modificando las entradas e identificar las opciones posibles para revertir una decisión no favorable. En esta actividad utilizaras la base de datos Heart Disease UCI Machine Learning Repository.

## Objetivos:
  - Aprender como obtener un contraejemplo dado un caso
  - Aprender a obtener un mejor contraejemplo usando el parámetro $\lambda$ de la fórmula de Watcher $\lambda (f_{\theta}(x')-y')+d(x^e-x')$
  - Aprender a aplicar en un caso de reversión de decisión las técnicas de contraejemplo e interpretar los resultados

## Para hacer
  - Realiza los cambios en el código necesarios de acuerdo a las instrucciones de la actividad y responde a las preguntas que se indican en dichas instrucciones.

In [9]:
# Instalación de las librerias
!pip install ucimlrepo
!pip install mlxtend

In [5]:
# Lectura de los datos desde el repositorio UCI
from ucimlrepo import fetch_ucirepo
import pandas as pd

# fetch dataset
heart_disease = fetch_ucirepo(id=45)

# data (as pandas dataframes)
dfX = heart_disease.data.features
dfY = heart_disease.data.targets
df = pd.concat([dfX, dfY], axis=1)
df = df.dropna()
dfX = df[['age','trestbps','chol','thalach','oldpeak','ca']]
dfY = df['num']
dfX.head()

,age,trestbps,chol,thalach,oldpeak,ca
0,63,145,233,150,2.3,0.0
1,67,160,286,108,1.5,3.0
2,67,120,229,129,2.6,2.0
3,37,130,250,187,3.5,0.0
4,41,130,204,172,1.4,0.0


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Aprendizaje del modelo random forest
X = dfX.values
y = np.ravel(dfY.values)

clf = RandomForestClassifier(random_state=0)
clf.fit(X, y)

RandomForestClassifier(random_state=0)

In [47]:
from mlxtend.evaluate import create_counterfactual

# Predicción para los ejemplos
TO_PREDICT = [2, 55, 65]

for indice in TO_PREDICT:

    x_ref = X[indice]

    print('True label:', y[55])
    print('Predicted label:', clf.predict(x_ref.reshape(1, -1))[0])
    print('Predicted probas:', clf.predict_proba(x_ref.reshape(1, -1)))

True label: 1
Predicted label: 1
Predicted probas: [[0.06 0.79 0.07 0.04 0.04]]
True label: 1
Predicted label: 1
Predicted probas: [[0.02 0.72 0.07 0.19 0.  ]]
True label: 1
Predicted label: 2
Predicted probas: [[0.03 0.06 0.82 0.07 0.02]]


In [44]:
threshold = 0.8
desired_class = 0
desired_proba = 1.0
original_prob = clf.predict_proba(x_ref.reshape(1, -1))[0][desired_class]

x_ref = X[55]

for lambda_value in range(1, 21, 1):
    # Búsqueda de un contraejemplo
    res = create_counterfactual(x_reference=x_ref,
                                y_desired=desired_class,
                                model=clf,
                                X_dataset=X,
                                y_desired_proba=desired_proba,
                                lammbda=lambda_value, #  hyperparameter
                                random_seed=123)
    
    # Ajustar el número de decimales
    for i in range(np.shape(res)[0]):
        if i==5:
            res[i] = round(res[i], 1)
        else:
            res[i] = round(res[i], 0)

    # Condición de parada
    predicted_prob = clf.predict_proba(res.reshape(1, -1))[0][desired_class]
    if predicted_prob >= threshold:
        break
    
print(f"Lambda value: {lambda_value}")

print('Features of the 55th example:', x_ref)
print('Features of the countefactual:', res)

print('Predictions for counterfactual:\n')
predicted_label = clf.predict(res.reshape(1, -1))[0]
assert predicted_label == 0
print('Predicted label:', predicted_label)

Predicted_probas = clf.predict_proba(res.reshape(1, -1))
print('Predicted probas:', Predicted_probas)

Lambda value: 2
Features of the 55th example: [ 54.  124.  266.  109.    2.2   1. ]
Features of the countefactual: [ 54. 124. 266. 155.  -0.   0.]
Predictions for counterfactual:

Predicted label: 0
Predicted probas: [[0.85 0.13 0.   0.02 0.  ]]


In [37]:
# Máximos y mínimos para comprobar si los valores están dentro de los rangos
print("Máximos:")
print(dfX.max())
print("\nMínimos:")
print(dfX.min())

Máximos
age          77.0
trestbps    200.0
chol        564.0
thalach     202.0
oldpeak       6.2
ca            3.0
dtype: float64

Mínimos
age          29.0
trestbps     94.0
chol        126.0
thalach      71.0
oldpeak       0.0
ca            0.0
dtype: float64


In [49]:
threshold = 0.8
desired_class = 0
desired_proba = 1.0

for indice in TO_PREDICT:
    for lambda_value in range(1, 21, 1):
        # Búsqueda de un contraejemplo
        res = create_counterfactual(x_reference=X[indice],
                                    y_desired=desired_class,
                                    model=clf,
                                    X_dataset=X,
                                    y_desired_proba=desired_proba,
                                    lammbda=lambda_value, #  hyperparameter
                                    random_seed=123)
        
        # Ajustar el número de decimales
        for i in range(np.shape(res)[0]):
            if i==5:
                res[i] = round(res[i], 1)
            else:
                res[i] = round(res[i], 0)

        
        predicted_prob = clf.predict_proba(res.reshape(1, -1))[0][desired_class]
        if predicted_prob >= threshold:
            break
    
    print(f"Indice: {indice}")
    print(f"Lambda value: {lambda_value}")

    print(f'Features of the {indice}th example:', X[indice])
    print('Features of the countefactual:', res)

    predicted_label = clf.predict(res.reshape(1, -1))[0]
    assert predicted_label == 0
    print('Predicted label:', predicted_label)

    predicted_probas = clf.predict_proba(res.reshape(1, -1))
    print('Predicted probas:', predicted_probas)

    print("-"*25)

Indice: 2
Lambda value: 13
Features of the 2th example: [ 67.  120.  229.  129.    2.6   2. ]
Features of the countefactual: [ 56. 120. 227. 148.   0.   0.]
Predicted label: 0
Predicted probas: [[0.94 0.06 0.   0.   0.  ]]
-------------------------
Indice: 55
Lambda value: 2
Features of the 55th example: [ 54.  124.  266.  109.    2.2   1. ]
Features of the countefactual: [ 54. 124. 266. 155.  -0.   0.]
Predicted label: 0
Predicted probas: [[0.85 0.13 0.   0.02 0.  ]]
-------------------------
Indice: 65
Lambda value: 9
Features of the 65th example: [ 60.  145.  282.  142.    2.8   2. ]
Features of the countefactual: [ 60. 139. 199. 143.  -0.   0.]
Predicted label: 0
Predicted probas: [[0.83 0.14 0.02 0.   0.01]]
-------------------------


In [59]:
# Contraejemplos de forma manual solo cambiando 2 características
casos = [
    [67, 120, 229, 129, 2.6, 2],
    [54, 124, 266, 109, .2, 1],
    [60, 145, 282, 142, 2.8, 2]
]

for caso in casos:
    print(f"Caso: {caso}")
    for _1 in range(0, 31, 1):
        for _2 in range(0, 4, 1):
            the_new = caso.copy()
            the_new[-2] = _1/10
            the_new[-1] = _2

            print(
                str(clf.predict_proba(np.array(the_new).reshape(1, -1))[0][0]).replace(".", ","),
                end = "\t"
            )
        print("")
    print("\n\n")

0,34	0,45	0,3	0,27	
0,35	0,45	0,31	0,28	
0,37	0,45	0,32	0,28	
0,35	0,45	0,32	0,28	
0,36	0,43	0,31	0,28	
0,38	0,4	0,27	0,25	
0,35	0,33	0,24	0,22	
0,35	0,33	0,24	0,22	
0,28	0,3	0,21	0,2	
0,28	0,23	0,16	0,14	
0,24	0,22	0,16	0,14	
0,27	0,22	0,16	0,14	
0,29	0,22	0,16	0,15	
0,29	0,22	0,16	0,15	
0,3	0,21	0,15	0,14	
0,3	0,25	0,18	0,17	
0,29	0,24	0,17	0,16	
0,28	0,21	0,17	0,14	
0,28	0,21	0,17	0,14	
0,29	0,18	0,14	0,12	
0,28	0,11	0,08	0,07	
0,28	0,11	0,08	0,07	
0,28	0,09	0,06	0,06	
0,29	0,1	0,07	0,06	
0,3	0,1	0,05	0,04	
0,26	0,08	0,04	0,03	
0,27	0,07	0,03	0,02	
0,28	0,07	0,03	0,02	
0,28	0,07	0,03	0,02	
0,27	0,06	0,02	0,01	
0,27	0,06	0,02	0,01	
